# 2. Métricas para evaluar clasificación

## Receiver Operating Characteristics (ROC)

Una curva ROC es una gráfica que permite evaluar el desempeño de un clasificador binario a medida que el umbral de clasificación es variado. la curva ROC se obtiene graficando la tasa de verdaderos positivos \textit{true positive rate (TPR)} versus la tasa de falsos positivos a diferentes configuraciones de umbrales. La tasa de positivos correctos también es conocida como sensibilidad, recall o probabilidad de detección. La tasa de falsos opsitivos es conocida como fall-out o falsa alarma. También se puede pensar de esta gráfica como una función del error tipo I de una regla de decisión. La curva ROC es entonces la sensibilidad en función del fall-out. 

En el caso de clasificación binaria, la predicción de clases para cada instancia en general está basada en una variable aleatoria continua $X$ que es el "score" calculado. Dado un parámetro de umbral $T$, la instancia es clasificada como positiva si $X>T$ y negativa de lo contrario. Entonces $X$ sigue una función de densidad de probabilidad $f_1(x)$ si la instancia pertenece realmente a la clase positiva y $f_0(x)$ de lo contrario. Entonces la tasa de positivos correctos está entonces dada por $TPR(T)=\int{T}^{\infty}f_1(x)dx$ y la tasa de falsos positivos entonces está dada por $FPR(T)=\int{T}^{\infty}f_0(x)dx$. La curva ROC grafica paramétricamente TPR(T) en función de FPR(T) a medida que (T) varía. 


## Configuración para multiclase

Las curvas ROC son tipicamente utilizadas en clasificación binaria para estudiar la salida de un clasificador. En orden de extender la idea de curva ROC y área AUC a una aplicación multiclase, es necesario reducir el problema a un problema de dos clases. Una curva puede ser dibujada para cada etiqueta, pero también se puede dibujar una curva considerando cada elemento del indicador de la matriz como un predictor binario (micro-promediando). 
